In [1]:
! git clone https://github.com/sigtot/unet-auto unet

fatal: destination path 'unet' already exists and is not an empty directory.


In [2]:
! git -C /content/unet checkout dev

fatal: cannot change to '/content/unet': No such file or directory


In [3]:
! git -C /content/unet pull

fatal: cannot change to '/content/unet': No such file or directory


In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [1]:
%load_ext autoreload
%autoreload 2
import os

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Cityscapes
from torchvision.utils import save_image
from unet import PavelNet
from unet import SigurdModel
import datetime

In [2]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# make a folder to save output images
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')
    

# a function used to transform numpy array to image format
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), x.size(1), x.size(2), x.size(3))
    return x

#################### select your hyperparameters ############################
num_epochs = 200
batch_size = 10
n_samples = 10

####### define image transforms, you can have other choices, explore it! #####
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# download dataset
# dataset_folder = '/content/drive/My Drive/citysmall'
dataset_folder = './data/cityscapes'
dataset = Cityscapes(dataset_folder, transform=transform, target_transform=transform, target_type='color')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

In [3]:
model = SigurdModel()
model.to(device)

SigurdModel(
  (discriminator): ArtNet(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (3): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (4): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (5): Conv2

In [4]:
model_save_path = "/content/drive/My Drive/models/model.pt"

In [5]:
refresh_model = True
if os.path.isfile(model_save_path) and not refresh_model:
    checkpoint = torch.load(model_save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Loaded {start_epoch + 1} epochs from file")
else:
    start_epoch = 0
    print("Starting with a fresh model")

Starting with a fresh model


In [6]:
for epoch in range(start_epoch, num_epochs):
    for i, data in enumerate(dataloader):
        if i > n_samples:
            break
        img, mask_with_alpha = data
        mask = mask_with_alpha[:, :3, :, :]
        img = img.to(device)
        mask = mask.to(device)
        #img = img.view(img.size(0), -1)
        
        img_pred = model.forward(mask, img)
        loss = model.backward()
    # ===================user interaction========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss))
    if epoch % 1 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, model_save_path)

        pic = to_img(img_pred.data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))
        
        ori_pic = to_img(img.data)
        save_image(ori_pic, './mlp_img/ori_image_{}.png'.format(epoch))
        
        mask_pic = to_img(mask.data)
        save_image(mask_pic, './mlp_img/mask_image_{}.png'.format(epoch))


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.